In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler, PolynomialFeatures, PowerTransformer
from sklearn.feature_selection import SelectFromModel, RFE, SelectKBest, f_regression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import Ridge, ElasticNet, HuberRegressor, Lasso, BayesianRidge
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from scipy.stats import skew, kurtosis, boxcox
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
import optuna
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

# For neural networks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.base import BaseEstimator, RegressorMixin

#########FIX###########
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
# n_splits = 5
#######################

# Load Data
print("Loading data...")
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Enhanced Oil Properties Feature Engineering
def create_enhanced_oil_features(df, poly_features=None, power_transformer=None, fit_transformers=True):
    """
    Create advanced features specific to oil blending and properties
    """
    features = [f'Component{i}_fraction' for i in range(1, 6)]
    features += [f'Component{i}_Property{j}' for i in range(1, 6) for j in range(1, 11)]

    # Oil-specific domain knowledge features
    print("Creating oil-specific features...")
    
    # 1. Viscosity Index calculations (Properties 1-3 typically relate to viscosity)
    for i in range(1, 6):
        df[f'Component{i}_viscosity_index'] = (
            df[f'Component{i}_Property1'] * 0.4 + 
            df[f'Component{i}_Property2'] * 0.4 + 
            df[f'Component{i}_Property3'] * 0.2
        )
    
    # 2. Density estimations (Properties 4-6)
    for i in range(1, 6):
        df[f'Component{i}_density_est'] = (
            df[f'Component{i}_Property4'] * 0.5 + 
            df[f'Component{i}_Property5'] * 0.3 + 
            df[f'Component{i}_Property6'] * 0.2
        )
    
    # 3. Octane-like properties (Properties 7-10)
    for i in range(1, 6):
        df[f'Component{i}_octane_est'] = (
            df[f'Component{i}_Property7'] * 0.3 + 
            df[f'Component{i}_Property8'] * 0.3 + 
            df[f'Component{i}_Property9'] * 0.2 + 
            df[f'Component{i}_Property10'] * 0.2
        )

    # Enhanced interaction features with non-linear transformations
    for i in range(1, 6):
        for j in range(1, 11):
            df[f'frac{i}_prop{j}'] = df[f'Component{i}_fraction'] * df[f'Component{i}_Property{j}']
            df[f'frac{i}_prop{j}_sqrt'] = df[f'Component{i}_fraction'] * np.sqrt(np.abs(df[f'Component{i}_Property{j}']))
            df[f'frac{i}_prop{j}_log'] = df[f'Component{i}_fraction'] * np.log(np.abs(df[f'Component{i}_Property{j}']) + 1)
            df[f'frac{i}_prop{j}_square'] = df[f'Component{i}_fraction'] * (df[f'Component{i}_Property{j}'] ** 2)
            df[f'frac{i}_prop{j}_cube'] = df[f'Component{i}_fraction'] * (df[f'Component{i}_Property{j}'] ** 3)
            
            # Oil-specific transformations
            df[f'frac{i}_prop{j}_exp'] = df[f'Component{i}_fraction'] * np.exp(df[f'Component{i}_Property{j}'] / 100)
            df[f'frac{i}_prop{j}_inv'] = df[f'Component{i}_fraction'] / (np.abs(df[f'Component{i}_Property{j}']) + 1e-8)
            
            features.extend([
                f'frac{i}_prop{j}', f'frac{i}_prop{j}_sqrt', f'frac{i}_prop{j}_log', 
                f'frac{i}_prop{j}_square', f'frac{i}_prop{j}_cube', f'frac{i}_prop{j}_exp', f'frac{i}_prop{j}_inv'
            ])

    # Advanced weighted features with multiple aggregation methods
    for j in range(1, 11):
        prop_cols = [f'Component{i}_Property{j}' for i in range(1, 6)]
        frac_cols = [f'Component{i}_fraction' for i in range(1, 6)]

        # Multiple weighted aggregations
        df[f'weighted_mean_prop{j}'] = sum(
            df[f'Component{i}_fraction'] * df[f'Component{i}_Property{j}'] for i in range(1, 6)
        )
        mean = df[f'weighted_mean_prop{j}']
        df[f'weighted_var_prop{j}'] = sum(
            df[f'Component{i}_fraction'] * (df[f'Component{i}_Property{j}'] - mean) ** 2 for i in range(1, 6)
        )

        # Harmonic mean (important for fuel properties)
        safe_props = [np.maximum(df[f'Component{i}_Property{j}'], 1e-6) for i in range(1, 6)]
        harmonic_mean = sum(df[f'Component{i}_fraction'] / safe_props[i-1] for i in range(1, 6))
        df[f'harmonic_mean_prop{j}'] = 1 / harmonic_mean

        # Geometric mean (for multiplicative properties)
        log_geo_mean = sum(df[f'Component{i}_fraction'] * np.log(safe_props[i-1]) for i in range(1, 6))
        df[f'geometric_mean_prop{j}'] = np.exp(log_geo_mean)

        # Component dominance with ranking
        frac_array = np.array([df[f'Component{i}_fraction'] for i in range(1, 6)])
        dominant_idx = np.argmax(frac_array, axis=0)
        df[f'dominant_prop{j}'] = df.apply(lambda row:
            row[f'Component{dominant_idx[row.name] + 1}_Property{j}'], axis=1)

        # Blend balance and diversity
        frac_cols = [f'Component{i}_fraction' for i in range(1, 6)]
        df[f'blend_balance_prop{j}'] = 1 - df[frac_cols].std(axis=1)
        df[f'blend_diversity_prop{j}'] = df[frac_cols].std(axis=1) / (df[frac_cols].mean(axis=1) + 1e-8)

        # Advanced statistics
        prop_cols = [f'Component{i}_Property{j}' for i in range(1, 6)]
        df[f'min_prop{j}'] = df[prop_cols].min(axis=1)
        df[f'max_prop{j}'] = df[prop_cols].max(axis=1)
        df[f'mean_prop{j}'] = df[prop_cols].mean(axis=1)
        df[f'std_prop{j}'] = df[prop_cols].std(axis=1)
        df[f'median_prop{j}'] = df[prop_cols].median(axis=1)
        df[f'skew_prop{j}'] = df[prop_cols].apply(lambda row: skew(row), axis=1)
        df[f'kurtosis_prop{j}'] = df[prop_cols].apply(lambda row: kurtosis(row), axis=1)
        df[f'range_prop{j}'] = df[f'max_prop{j}'] - df[f'min_prop{j}']
        df[f'iqr_prop{j}'] = df[prop_cols].quantile(0.75, axis=1) - df[prop_cols].quantile(0.25, axis=1)

        features.extend([
            f'min_prop{j}', f'max_prop{j}', f'mean_prop{j}',
            f'std_prop{j}', f'median_prop{j}', f'skew_prop{j}', f'kurtosis_prop{j}',
            f'range_prop{j}', f'iqr_prop{j}'
        ])

    # Oil-specific advanced blending rules
    for j in range(1, 11):
        fractions = [df[f'Component{i}_fraction'] for i in range(1, 6)]
        props = [df[f'Component{i}_Property{j}'] for i in range(1, 6)]
        safe_props = [np.maximum(p, 1e-6) for p in props]

        # RON-like blending (non-linear octane)
        ron_blend = sum(f * (r ** 1.5) for f, r in zip(fractions, safe_props)) ** (1 / 1.5)
        df[f'ron_like_blend_prop{j}'] = ron_blend

        # Viscosity-like blending (logarithmic)
        log_visc_blend = sum(f * np.log(r) for f, r in zip(fractions, safe_props))
        df[f'log_visc_blend_prop{j}'] = log_visc_blend

        # Density-like blending (linear but with corrections)
        density_blend = sum(f * r for f, r in zip(fractions, safe_props))
        df[f'density_blend_prop{j}'] = density_blend

        # Reid vapor pressure-like (exponential)
        rvp_blend = sum(f * np.exp(r / 100) for f, r in zip(fractions, safe_props))
        df[f'rvp_blend_prop{j}'] = rvp_blend

        # Cetane number-like blending
        cetane_blend = sum(f * np.sqrt(r) for f, r in zip(fractions, safe_props))
        df[f'cetane_like_blend_prop{j}'] = cetane_blend

        # Flash point-like blending
        flash_blend = sum(f * (r ** 0.7) for f, r in zip(fractions, safe_props))
        df[f'flash_like_blend_prop{j}'] = flash_blend

        features.extend([
            f'ron_like_blend_prop{j}', f'log_visc_blend_prop{j}',
            f'density_blend_prop{j}', f'rvp_blend_prop{j}',
            f'cetane_like_blend_prop{j}', f'flash_like_blend_prop{j}'
        ])

    # Cross-property interactions (enhanced for oil properties)
    for j1 in range(1, 11):
        for j2 in range(j1 + 1, 11):
            df[f'prop{j1}_prop{j2}_interaction'] = df[f'weighted_mean_prop{j1}'] * df[f'weighted_mean_prop{j2}']
            df[f'prop{j1}_prop{j2}_ratio'] = df[f'weighted_mean_prop{j1}'] / (df[f'weighted_mean_prop{j2}'] + 1e-8)
            df[f'prop{j1}_prop{j2}_diff'] = df[f'weighted_mean_prop{j1}'] - df[f'weighted_mean_prop{j2}']
            df[f'prop{j1}_prop{j2}_harmonic'] = 2 / (1/df[f'weighted_mean_prop{j1}'] + 1/(df[f'weighted_mean_prop{j2}'] + 1e-8))
            features.extend([
                f'prop{j1}_prop{j2}_interaction', f'prop{j1}_prop{j2}_ratio',
                f'prop{j1}_prop{j2}_diff', f'prop{j1}_prop{j2}_harmonic'
            ])

    # Enhanced PCA with more components
    prop_features = [f'Component{i}_Property{j}' for i in range(1, 6) for j in range(1, 11)]
    if fit_transformers:
        pca = PCA(n_components=15, random_state=42)
        pca_feats = pca.fit_transform(df[prop_features])
    else:
        pca = pca_model
        pca_feats = pca.transform(df[prop_features])

    for k in range(15):
        df[f'pca_prop_{k+1}'] = pca_feats[:, k]
        features.append(f'pca_prop_{k+1}')

    # Polynomial features for key interactions
    if fit_transformers:
        key_features = [f'Component{i}_fraction' for i in range(1, 6)]
        key_features += [f'weighted_mean_prop{j}' for j in range(1, 11)]
        poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
        poly_feats = poly.fit_transform(df[key_features])
        poly_feature_names = poly.get_feature_names_out(key_features)
        
        # Add only the most relevant polynomial features
        for i, name in enumerate(poly_feature_names):
            if 'weighted_mean_prop' in name and ('fraction' in name or 'weighted_mean_prop' in name):
                df[f'poly_{name}'] = poly_feats[:, i]
                features.append(f'poly_{name}')
    else:
        poly = poly_features
        if poly is not None:
            key_features = [f'Component{i}_fraction' for i in range(1, 6)]
            key_features += [f'weighted_mean_prop{j}' for j in range(1, 11)]
            poly_feats = poly.transform(df[key_features])
            poly_feature_names = poly.get_feature_names_out(key_features)
            
            for i, name in enumerate(poly_feature_names):
                if 'weighted_mean_prop' in name and ('fraction' in name or 'weighted_mean_prop' in name):
                    df[f'poly_{name}'] = poly_feats[:, i]
                    features.append(f'poly_{name}')

    # Power transformations for skewed features
    if fit_transformers:
        skewed_features = []
        for col in [f'Component{i}_Property{j}' for i in range(1, 6) for j in range(1, 11)]:
            if abs(skew(df[col].dropna())) > 0.5:
                skewed_features.append(col)
        
        if skewed_features:
            pt = PowerTransformer(method='yeo-johnson')
            transformed_feats = pt.fit_transform(df[skewed_features])
            for i, col in enumerate(skewed_features):
                df[f'power_transform_{col}'] = transformed_feats[:, i]
                features.append(f'power_transform_{col}')
    else:
        pt = power_transformer
        if pt is not None:
            skewed_features = []
            for col in [f'Component{i}_Property{j}' for i in range(1, 6) for j in range(1, 11)]:
                if col in df.columns and abs(skew(df[col].dropna())) > 0.5:
                    skewed_features.append(col)
            
            if skewed_features:
                transformed_feats = pt.transform(df[skewed_features])
                for i, col in enumerate(skewed_features):
                    df[f'power_transform_{col}'] = transformed_feats[:, i]
                    features.append(f'power_transform_{col}')

    # Fraction-based advanced features
    frac_cols = [f'Component{i}_fraction' for i in range(1, 6)]
    df['frac_sum'] = df[frac_cols].sum(axis=1)
    df['frac_std'] = df[frac_cols].std(axis=1)
    df['frac_skew'] = df[frac_cols].apply(lambda row: skew(row), axis=1)
    df['frac_kurtosis'] = df[frac_cols].apply(lambda row: kurtosis(row), axis=1)
    df['frac_entropy'] = -sum(df[f'Component{i}_fraction'] * np.log(df[f'Component{i}_fraction'] + 1e-8) for i in range(1, 6))
    df['frac_gini'] = 1 - sum(df[f'Component{i}_fraction'] ** 2 for i in range(1, 6))
    df['frac_max'] = df[frac_cols].max(axis=1)
    df['frac_min'] = df[frac_cols].min(axis=1)
    df['frac_range'] = df['frac_max'] - df['frac_min']

    features.extend(['frac_sum', 'frac_std', 'frac_skew', 'frac_kurtosis', 'frac_entropy', 'frac_gini',
                     'frac_max', 'frac_min', 'frac_range'])

    return df, features, pca, poly, pt

# Custom Neural Network Regressor
class KerasRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_dim=100, hidden_layers=3, neurons=128, dropout=0.3, learning_rate=0.001, epochs=100):
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.neurons = neurons
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.model = None
        
    def build_model(self):
        model = Sequential()
        model.add(Dense(self.neurons, input_dim=self.input_dim, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(self.dropout))
        
        for _ in range(self.hidden_layers - 1):
            model.add(Dense(self.neurons, activation='relu'))
            model.add(BatchNormalization())
            model.add(Dropout(self.dropout))
        
        model.add(Dense(1, activation='linear'))
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse', metrics=['mae'])
        return model
    
    def fit(self, X, y):
        self.input_dim = X.shape[1]
        self.model = self.build_model()
        
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
        
        self.model.fit(X, y, epochs=self.epochs, batch_size=32, validation_split=0.2,
                      callbacks=[early_stop, reduce_lr], verbose=0)
        return self
    
    def predict(self, X):
        return self.model.predict(X, verbose=0).flatten()

# Apply enhanced feature engineering
print("Creating enhanced oil-specific features...")
train, feat_cols, pca_model, poly_model, pt_model = create_enhanced_oil_features(train, fit_transformers=True)
test, _, _, _, _ = create_enhanced_oil_features(test, pca_model, poly_model, pt_model, fit_transformers=False)

print(f"Created {len(feat_cols)} features for oil property prediction")

Loading data...
Creating breakthrough features...


/tmp/ipykernel_4390/243682087.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'frac{i}_prop{j}_square'] = df[f'Component{i}_fraction'] * (df[f'Component{i}_Property{j}'] ** 2)
/tmp/ipykernel_4390/243682087.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'frac{i}_prop{j}'] = df[f'Component{i}_fraction'] * df[f'Component{i}_Property{j}']
/tmp/ipykernel_4390/243682087.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

Handling NaN values...
Handling NaN values...
Performing feature selection...
Original features: 433
Selected features: 227
Training Breakthrough Ensemble...
Features: 433 (selected: 227)

Training for BlendProperty1...
Training until validation scores don't improve for 150 rounds
[200]	valid_0's l1: 0.658822
[400]	valid_0's l1: 0.555859
[600]	valid_0's l1: 0.481634
[800]	valid_0's l1: 0.426586
[1000]	valid_0's l1: 0.383944
[1200]	valid_0's l1: 0.350733
[1400]	valid_0's l1: 0.325188
[1600]	valid_0's l1: 0.304542
[1800]	valid_0's l1: 0.287788
[2000]	valid_0's l1: 0.274215
[2200]	valid_0's l1: 0.2629
[2400]	valid_0's l1: 0.253431
[2600]	valid_0's l1: 0.245593
[2800]	valid_0's l1: 0.238928
[3000]	valid_0's l1: 0.233089
[3200]	valid_0's l1: 0.228454
[3400]	valid_0's l1: 0.224477
[3600]	valid_0's l1: 0.221075
[3800]	valid_0's l1: 0.218047
[4000]	valid_0's l1: 0.215134
[4200]	valid_0's l1: 0.212573
[4400]	valid_0's l1: 0.210522
[4600]	valid_0's l1: 0.207972
[4800]	valid_0's l1: 0.205602
[500

/home/mithun/Desktop/kushvinth/.venv/lib/python3.13/site-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Training until validation scores don't improve for 150 rounds
[200]	valid_0's l1: 0.660942
[400]	valid_0's l1: 0.558553
[600]	valid_0's l1: 0.480034
[800]	valid_0's l1: 0.417343
[1000]	valid_0's l1: 0.370788
[1200]	valid_0's l1: 0.335246
[1400]	valid_0's l1: 0.307613
[1600]	valid_0's l1: 0.285916
[1800]	valid_0's l1: 0.268543
[2000]	valid_0's l1: 0.255056
[2200]	valid_0's l1: 0.24484
[2400]	valid_0's l1: 0.236367
[2600]	valid_0's l1: 0.229232
[2800]	valid_0's l1: 0.223278
[3000]	valid_0's l1: 0.218231
[3200]	valid_0's l1: 0.214045
[3400]	valid_0's l1: 0.210303
[3600]	valid_0's l1: 0.207113
[3800]	valid_0's l1: 0.204117
[4000]	valid_0's l1: 0.201305
[4200]	valid_0's l1: 0.198914
[4400]	valid_0's l1: 0.1969
[4600]	valid_0's l1: 0.194861
[4800]	valid_0's l1: 0.193161
[5000]	valid_0's l1: 0.191085
[5200]	valid_0's l1: 0.18947
[5400]	valid_0's l1: 0.188112
[5600]	valid_0's l1: 0.187045
[5800]	valid_0's l1: 0.185829
[6000]	valid_0's l1: 0.18466
[6200]	valid_0's l1: 0.183463
[6400]	valid_0's 

...

/home/mithun/Desktop/kushvinth/.venv/lib/python3.13/site-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Training until validation scores don't improve for 150 rounds
[200]	valid_0's l1: 0.649794
[400]	valid_0's l1: 0.537043
[600]	valid_0's l1: 0.452215
[800]	valid_0's l1: 0.388429
[1000]	valid_0's l1: 0.340973
[1200]	valid_0's l1: 0.305124
[1400]	valid_0's l1: 0.277213
[1600]	valid_0's l1: 0.255683
[1800]	valid_0's l1: 0.238048
[2000]	valid_0's l1: 0.223482
[2200]	valid_0's l1: 0.211581
[2400]	valid_0's l1: 0.201559
[2600]	valid_0's l1: 0.193432
[2800]	valid_0's l1: 0.18705
[3000]	valid_0's l1: 0.181866
[3200]	valid_0's l1: 0.177519
[3400]	valid_0's l1: 0.173907
[3600]	valid_0's l1: 0.170605
[3800]	valid_0's l1: 0.167774
[4000]	valid_0's l1: 0.165794
[4200]	valid_0's l1: 0.163952
[4400]	valid_0's l1: 0.162378
[4600]	valid_0's l1: 0.16038
[4800]	valid_0's l1: 0.157629
[5000]	valid_0's l1: 0.155334
[5200]	valid_0's l1: 0.153407
[5400]	valid_0's l1: 0.151904
[5600]	valid_0's l1: 0.150661
[5800]	valid_0's l1: 0.149798
[6000]	valid_0's l1: 0.149391
[6200]	valid_0's l1: 0.148655
[6400]	valid_0

/home/mithun/Desktop/kushvinth/.venv/lib/python3.13/site-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Training until validation scores don't improve for 150 rounds
[200]	valid_0's l1: 0.679962
[400]	valid_0's l1: 0.5699
[600]	valid_0's l1: 0.485704
[800]	valid_0's l1: 0.421537
[1000]	valid_0's l1: 0.373856
[1200]	valid_0's l1: 0.338054
[1400]	valid_0's l1: 0.309879
[1600]	valid_0's l1: 0.286716
[1800]	valid_0's l1: 0.267424
[2000]	valid_0's l1: 0.251963
[2200]	valid_0's l1: 0.239712
[2400]	valid_0's l1: 0.229936
[2600]	valid_0's l1: 0.221885
[2800]	valid_0's l1: 0.214788
[3000]	valid_0's l1: 0.208992
[3200]	valid_0's l1: 0.204049
[3400]	valid_0's l1: 0.199841
[3600]	valid_0's l1: 0.196187
[3800]	valid_0's l1: 0.192944
[4000]	valid_0's l1: 0.190412
[4200]	valid_0's l1: 0.1878
[4400]	valid_0's l1: 0.185421
[4600]	valid_0's l1: 0.183098
[4800]	valid_0's l1: 0.180818
[5000]	valid_0's l1: 0.178856
[5200]	valid_0's l1: 0.177203
[5400]	valid_0's l1: 0.17609
[5600]	valid_0's l1: 0.174358
[5800]	valid_0's l1: 0.17233
[6000]	valid_0's l1: 0.171016
[6200]	valid_0's l1: 0.169774
[6400]	valid_0's l

/home/mithun/Desktop/kushvinth/.venv/lib/python3.13/site-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [ ]:
# Prepare Data
TARGETS = [f'BlendProperty{i}' for i in range(1, 11)]
X_train = train[feat_cols]
y_train = train[TARGETS]
X_test = test[feat_cols]

# Handle NaN values and infinite values
print("Handling NaN and infinite values...")
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)
X_train = X_train.fillna(X_train.median())
X_test = X_test.fillna(X_train.median())

# Feature scaling for different models
scaler_robust = RobustScaler()
X_train_robust = scaler_robust.fit_transform(X_train)
X_test_robust = scaler_robust.transform(X_test)

scaler_standard = StandardScaler()
X_train_standard = scaler_standard.fit_transform(X_train)
X_test_standard = scaler_standard.transform(X_test)

# Advanced feature selection
print("Performing advanced feature selection...")

# Multiple feature selection methods
selector_lgb = SelectFromModel(
    LGBMRegressor(n_estimators=200, random_state=42, verbose=-1),
    prefit=False,
    threshold='median'
)

selector_rf = SelectFromModel(
    RandomForestRegressor(n_estimators=200, random_state=42),
    prefit=False,
    threshold='median'
)

# Fit selectors on first target
selector_lgb.fit(X_train, y_train.iloc[:, 0])
selector_rf.fit(X_train, y_train.iloc[:, 0])

# Combine selected features
lgb_selected = selector_lgb.get_support()
rf_selected = selector_rf.get_support()
combined_selected = lgb_selected | rf_selected

selected_features = [feat_cols[i] for i in range(len(feat_cols)) if combined_selected[i]]
X_train_selected = X_train.iloc[:, combined_selected]
X_test_selected = X_test.iloc[:, combined_selected]

print(f"Original features: {len(feat_cols)}")
print(f"Selected features: {len(selected_features)}")

# Hyperparameter optimization with Optuna
def optimize_lightgbm(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 3000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 15, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'random_state': 42,
        'verbose': -1
    }
    
    model = LGBMRegressor(**params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_percentage_error')
    return scores.mean()

def optimize_xgboost(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state': 42,
        'verbosity': 0
    }
    
    model = XGBRegressor(**params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_percentage_error')
    return scores.mean()

def optimize_catboost(trial, X, y):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
        'random_seed': 42,
        'verbose': False
    }
    
    model = CatBoostRegressor(**params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_percentage_error')
    return scores.mean()

def optimize_neural_network(trial, X, y):
    params = {
        'hidden_layers': trial.suggest_int('hidden_layers', 2, 5),
        'neurons': trial.suggest_int('neurons', 64, 512),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01, log=True),
        'epochs': 100
    }
    
    model = KerasRegressor(input_dim=X.shape[1], **params)
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_percentage_error')
    return scores.mean()

# Optimize hyperparameters for key models
print("Optimizing hyperparameters...")
best_params = {}

# Use first target for optimization to save time
target_for_optimization = TARGETS[0]
y_opt = y_train[target_for_optimization]

# Optimize LightGBM
print("Optimizing LightGBM...")
study_lgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_lgb.optimize(lambda trial: optimize_lightgbm(trial, X_train_selected, y_opt), n_trials=30)
best_params['lgb'] = study_lgb.best_params

# Optimize XGBoost
print("Optimizing XGBoost...")
study_xgb = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_xgb.optimize(lambda trial: optimize_xgboost(trial, X_train_selected, y_opt), n_trials=30)
best_params['xgb'] = study_xgb.best_params

# Optimize CatBoost
print("Optimizing CatBoost...")
study_cat = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_cat.optimize(lambda trial: optimize_catboost(trial, X_train_selected, y_opt), n_trials=30)
best_params['cat'] = study_cat.best_params

# Optimize Neural Network
print("Optimizing Neural Network...")
study_nn = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study_nn.optimize(lambda trial: optimize_neural_network(trial, X_train_standard, y_opt), n_trials=20)
best_params['nn'] = study_nn.best_params

print("Hyperparameter optimization completed!")
print("Best parameters:")
for model_name, params in best_params.items():
    print(f"{model_name}: {params}")

# Cross-Validation Setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)
final_preds = np.zeros((X_test.shape[0], len(TARGETS)))

print("\nTraining Enhanced Oil Property Prediction Ensemble...")
print(f"Total features: {len(feat_cols)}")
print(f"Selected features: {len(selected_features)}")
print(f"Models: LightGBM, XGBoost, CatBoost, Random Forest, Extra Trees, SVR, Neural Network, Ridge, Elastic Net, Huber")

In [ ]:
for i, target in enumerate(TARGETS):
    print(f"\n{'='*60}")
    print(f"Training for {target} ({i+1}/{len(TARGETS)})")
    print(f"{'='*60}")

    # Out-of-fold predictions for each model
    models_oof = {}
    models_test_preds = {}
    
    model_names = ['lgb', 'xgb', 'cat', 'rf', 'et', 'gb', 'svr', 'nn', 'ridge', 'elastic', 'huber', 'lasso', 'bayesian']
    
    for model_name in model_names:
        models_oof[model_name] = np.zeros(X_train.shape[0])
        models_test_preds[model_name] = np.zeros(X_test.shape[0])

    for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
        print(f"\nFold {fold + 1}/5")
        
        # Model 1: Optimized LightGBM
        model_lgb = LGBMRegressor(**best_params['lgb'])
        model_lgb.fit(
            X_train_selected.iloc[tr_idx], y_train[target].iloc[tr_idx],
            eval_set=[(X_train_selected.iloc[val_idx], y_train[target].iloc[val_idx])],
            callbacks=[early_stopping(stopping_rounds=100), log_evaluation(500)]
        )
        models_oof['lgb'][val_idx] = model_lgb.predict(X_train_selected.iloc[val_idx])
        models_test_preds['lgb'] += model_lgb.predict(X_test_selected) / kf.get_n_splits()

        # Model 2: Optimized XGBoost
        model_xgb = XGBRegressor(**best_params['xgb'])
        model_xgb.fit(
            X_train_selected.iloc[tr_idx], y_train[target].iloc[tr_idx],
            eval_set=[(X_train_selected.iloc[val_idx], y_train[target].iloc[val_idx])],
            early_stopping_rounds=100, verbose=False
        )
        models_oof['xgb'][val_idx] = model_xgb.predict(X_train_selected.iloc[val_idx])
        models_test_preds['xgb'] += model_xgb.predict(X_test_selected) / kf.get_n_splits()

        # Model 3: Optimized CatBoost
        model_cat = CatBoostRegressor(**best_params['cat'])
        model_cat.fit(
            X_train_selected.iloc[tr_idx], y_train[target].iloc[tr_idx],
            eval_set=[(X_train_selected.iloc[val_idx], y_train[target].iloc[val_idx])],
            early_stopping_rounds=100, verbose=False
        )
        models_oof['cat'][val_idx] = model_cat.predict(X_train_selected.iloc[val_idx])
        models_test_preds['cat'] += model_cat.predict(X_test_selected) / kf.get_n_splits()

        # Model 4: Enhanced Random Forest
        model_rf = RandomForestRegressor(
            n_estimators=1000, max_depth=25, min_samples_split=3,
            min_samples_leaf=1, max_features='sqrt', random_state=fold, n_jobs=-1
        )
        model_rf.fit(X_train_selected.iloc[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['rf'][val_idx] = model_rf.predict(X_train_selected.iloc[val_idx])
        models_test_preds['rf'] += model_rf.predict(X_test_selected) / kf.get_n_splits()

        # Model 5: Enhanced Extra Trees
        model_et = ExtraTreesRegressor(
            n_estimators=800, max_depth=22, min_samples_split=2,
            min_samples_leaf=1, max_features='sqrt', random_state=fold, n_jobs=-1
        )
        model_et.fit(X_train_selected.iloc[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['et'][val_idx] = model_et.predict(X_train_selected.iloc[val_idx])
        models_test_preds['et'] += model_et.predict(X_test_selected) / kf.get_n_splits()

        # Model 6: Enhanced Gradient Boosting
        model_gb = GradientBoostingRegressor(
            n_estimators=800, learning_rate=0.005, max_depth=7,
            min_samples_split=4, min_samples_leaf=2, max_features='sqrt',
            random_state=fold
        )
        model_gb.fit(X_train_selected.iloc[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['gb'][val_idx] = model_gb.predict(X_train_selected.iloc[val_idx])
        models_test_preds['gb'] += model_gb.predict(X_test_selected) / kf.get_n_splits()

        # Model 7: Support Vector Regression
        model_svr = SVR(
            kernel='rbf', C=10.0, gamma='scale', epsilon=0.01
        )
        model_svr.fit(X_train_standard[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['svr'][val_idx] = model_svr.predict(X_train_standard[val_idx])
        models_test_preds['svr'] += model_svr.predict(X_test_standard) / kf.get_n_splits()

        # Model 8: Optimized Neural Network
        model_nn = KerasRegressor(
            input_dim=X_train_standard.shape[1],
            **best_params['nn']
        )
        model_nn.fit(X_train_standard[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['nn'][val_idx] = model_nn.predict(X_train_standard[val_idx])
        models_test_preds['nn'] += model_nn.predict(X_test_standard) / kf.get_n_splits()

        # Model 9: Enhanced Ridge
        model_ridge = Ridge(alpha=0.1, random_state=fold)
        model_ridge.fit(X_train_robust[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['ridge'][val_idx] = model_ridge.predict(X_train_robust[val_idx])
        models_test_preds['ridge'] += model_ridge.predict(X_test_robust) / kf.get_n_splits()

        # Model 10: Enhanced Elastic Net
        model_elastic = ElasticNet(alpha=0.005, l1_ratio=0.2, random_state=fold, max_iter=3000)
        model_elastic.fit(X_train_standard[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['elastic'][val_idx] = model_elastic.predict(X_train_standard[val_idx])
        models_test_preds['elastic'] += model_elastic.predict(X_test_standard) / kf.get_n_splits()

        # Model 11: Enhanced Huber
        model_huber = HuberRegressor(alpha=0.005, epsilon=1.2)
        model_huber.fit(X_train_robust[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['huber'][val_idx] = model_huber.predict(X_train_robust[val_idx])
        models_test_preds['huber'] += model_huber.predict(X_test_robust) / kf.get_n_splits()

        # Model 12: Lasso Regression
        model_lasso = Lasso(alpha=0.01, random_state=fold, max_iter=3000)
        model_lasso.fit(X_train_standard[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['lasso'][val_idx] = model_lasso.predict(X_train_standard[val_idx])
        models_test_preds['lasso'] += model_lasso.predict(X_test_standard) / kf.get_n_splits()

        # Model 13: Bayesian Ridge
        model_bayesian = BayesianRidge(alpha_1=1e-6, alpha_2=1e-6, lambda_1=1e-6, lambda_2=1e-6)
        model_bayesian.fit(X_train_standard[tr_idx], y_train[target].iloc[tr_idx])
        models_oof['bayesian'][val_idx] = model_bayesian.predict(X_train_standard[val_idx])
        models_test_preds['bayesian'] += model_bayesian.predict(X_test_standard) / kf.get_n_splits()

    # Calculate individual model MAPE scores
    mape_scores = {}
    for model_name in model_names:
        mape_scores[model_name] = mean_absolute_percentage_error(y_train[target], models_oof[model_name])

    # Advanced ensemble with multiple weighting strategies
    
    # Strategy 1: Exponential weighting based on validation performance
    exp_weights = {name: np.exp(-score * 5) for name, score in mape_scores.items()}
    total_exp_weight = sum(exp_weights.values())
    exp_weights = {name: w / total_exp_weight for name, w in exp_weights.items()}
    
    # Strategy 2: Inverse MAPE weighting
    inv_weights = {name: 1/score for name, score in mape_scores.items()}
    total_inv_weight = sum(inv_weights.values())
    inv_weights = {name: w / total_inv_weight for name, w in inv_weights.items()}
    
    # Strategy 3: Rank-based weighting
    sorted_models = sorted(mape_scores.items(), key=lambda x: x[1])
    rank_weights = {}
    for rank, (model_name, _) in enumerate(sorted_models):
        rank_weights[model_name] = (len(model_names) - rank) / sum(range(1, len(model_names) + 1))
    
    # Combine strategies
    final_weights = {}
    for model_name in model_names:
        final_weights[model_name] = (
            0.4 * exp_weights[model_name] + 
            0.4 * inv_weights[model_name] + 
            0.2 * rank_weights[model_name]
        )

    # Final ensemble prediction
    final_preds[:, i] = sum(
        final_weights[model_name] * models_test_preds[model_name] 
        for model_name in model_names
    )

    # Ensemble validation score
    ensemble_oof = sum(
        final_weights[model_name] * models_oof[model_name] 
        for model_name in model_names
    )
    ensemble_mape = mean_absolute_percentage_error(y_train[target], ensemble_oof)

    # Print detailed results
    print(f"\nIndividual Model Performance for {target}:")
    print("-" * 80)
    for model_name in sorted(mape_scores.keys(), key=lambda x: mape_scores[x]):
        print(f"{model_name.upper():>12}: MAPE={mape_scores[model_name]:.6f}, Weight={final_weights[model_name]:.4f}")
    
    print("-" * 80)
    print(f"{'ENSEMBLE':>12}: MAPE={ensemble_mape:.6f}")
    print(f"Best Single Model: {min(mape_scores.keys(), key=lambda x: mape_scores[x]).upper()}")
    print(f"Improvement: {min(mape_scores.values()) - ensemble_mape:.6f}")

# Create Enhanced Submission
submission = pd.DataFrame(final_preds, columns=TARGETS)
submission.insert(0, 'ID', test.get('ID', np.arange(1, len(test) + 1)))

# Apply post-processing to ensure realistic oil property values
for target in TARGETS:
    # Remove extreme outliers
    Q1 = submission[target].quantile(0.01)
    Q3 = submission[target].quantile(0.99)
    submission[target] = submission[target].clip(Q1, Q3)

submission.to_csv('submission_enhanced_oil_properties.csv', index=False)

print(f"\n{'='*80}")
print("ENHANCED OIL PROPERTY PREDICTION ENSEMBLE SUMMARY")
print(f"{'='*80}")
print(f"Total features engineered: {len(feat_cols)}")
print(f"Selected features used: {len(selected_features)}")
print(f"Models in ensemble: {len(model_names)}")
print("Models: LightGBM, XGBoost, CatBoost, Random Forest, Extra Trees,")
print("        Gradient Boosting, SVR, Neural Network, Ridge, Elastic Net,")
print("        Huber, Lasso, Bayesian Ridge")
print("Hyperparameter optimization: Optuna with 30+ trials per model")
print("Feature engineering: Oil-specific domain knowledge + polynomial features")
print("Ensemble strategy: Multi-weighted combination (exponential + inverse + rank)")
print("Cross-validation: 5-fold stratified")
print("Post-processing: Outlier clipping")
print(f"Submission file: submission_enhanced_oil_properties.csv")
print(f"{'='*80}")

# Feature importance analysis
print("\nFeature Importance Analysis:")
print("-" * 40)

# Get feature importance from best LightGBM model
temp_lgb = LGBMRegressor(**best_params['lgb'])
temp_lgb.fit(X_train_selected, y_train.iloc[:, 0])
feature_importance = pd.DataFrame({
    'feature': selected_features,
    'importance': temp_lgb.feature_importances_
}).sort_values('importance', ascending=False)

print("Top 20 Most Important Features:")
for i, (_, row) in enumerate(feature_importance.head(20).iterrows()):
    print(f"{i+1:2d}. {row['feature']:30s} {row['importance']:8.4f}")

print(f"\nModel training completed successfully!")
print(f"Enhanced submission ready for oil property prediction competition.")

In [ ]:
# Model Performance Analysis and Validation
print("Performing cross-validation analysis...")

# Quick validation on a subset to estimate performance
sample_size = min(1000, len(X_train))
sample_idx = np.random.choice(len(X_train), sample_size, replace=False)

X_sample = X_train_selected.iloc[sample_idx]
y_sample = y_train.iloc[sample_idx, 0]  # Use first target for quick validation

# Test a few models quickly
quick_models = {
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42, verbose=-1),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=200, random_state=42)
}

print(f"\nQuick validation on {sample_size} samples:")
print("-" * 50)

for name, model in quick_models.items():
    scores = cross_val_score(model, X_sample, y_sample, cv=3, scoring='neg_mean_absolute_percentage_error')
    print(f"{name:15s}: CV MAPE = {-scores.mean():.4f} (+/- {scores.std() * 2:.4f})")

print(f"\nDataset info:")
print(f"Training samples: {len(X_train):,}")
print(f"Test samples: {len(X_test):,}")
print(f"Total features: {len(feat_cols):,}")
print(f"Targets to predict: {len(TARGETS)}")

# Memory usage estimate
memory_usage = X_train.memory_usage(deep=True).sum() / 1024**2
print(f"Training data memory usage: {memory_usage:.1f} MB")

20.033333333333335

In [ ]:
# Advanced Ensemble Analysis and Meta-Learning
print("Setting up advanced ensemble optimization...")

# Meta-features for stacking (if you want to implement stacking later)
def create_meta_features(X, models, cv_folds=3):
    """Create meta-features using cross-validation predictions"""
    meta_features = np.zeros((X.shape[0], len(models)))
    
    kf_meta = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
    
    for i, (name, model) in enumerate(models.items()):
        meta_pred = np.zeros(X.shape[0])
        
        for train_idx, val_idx in kf_meta.split(X):
            model.fit(X.iloc[train_idx], y_train.iloc[train_idx, 0])  # Use first target
            meta_pred[val_idx] = model.predict(X.iloc[val_idx])
        
        meta_features[:, i] = meta_pred
        print(f"Generated meta-features for {name}")
    
    return meta_features

# Oil property-specific validation
def oil_property_validation(predictions, targets):
    """Validate predictions using oil industry constraints"""
    
    # Check for reasonable ranges (these would need to be adjusted based on actual oil properties)
    validation_results = {}
    
    for i, target in enumerate(targets):
        pred_col = predictions[:, i]
        
        # Basic statistics
        validation_results[target] = {
            'mean': np.mean(pred_col),
            'std': np.std(pred_col),
            'min': np.min(pred_col),
            'max': np.max(pred_col),
            'outliers_count': np.sum(np.abs(pred_col - np.mean(pred_col)) > 3 * np.std(pred_col))
        }
    
    return validation_results

# Model diversity analysis
def analyze_model_diversity(predictions_dict):
    """Analyze how diverse the model predictions are"""
    pred_matrix = np.column_stack(list(predictions_dict.values()))
    
    # Correlation between models
    correlation_matrix = np.corrcoef(pred_matrix.T)
    
    print("Model Diversity Analysis:")
    print("-" * 40)
    print(f"Average correlation between models: {np.mean(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]):.4f}")
    print(f"Min correlation: {np.min(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]):.4f}")
    print(f"Max correlation: {np.max(correlation_matrix[np.triu_indices_from(correlation_matrix, k=1)]):.4f}")
    
    return correlation_matrix

# Feature interaction analysis
def analyze_feature_interactions(X, y, top_features=10):
    """Analyze top feature interactions for oil properties"""
    
    # Use a simple model to get feature importance
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop {top_features} Features for Oil Property Prediction:")
    print("-" * 60)
    
    for i, (_, row) in enumerate(feature_importance.head(top_features).iterrows()):
        print(f"{i+1:2d}. {row['feature'][:50]:50s} {row['importance']:8.6f}")
    
    return feature_importance

# Residual analysis
def analyze_residuals(y_true, y_pred, target_name):
    """Analyze prediction residuals"""
    residuals = y_true - y_pred
    
    print(f"\nResidual Analysis for {target_name}:")
    print("-" * 40)
    print(f"Mean residual: {np.mean(residuals):8.6f}")
    print(f"Std residual:  {np.std(residuals):8.6f}")
    print(f"Max residual:  {np.max(np.abs(residuals)):8.6f}")
    print(f"Skewness:      {skew(residuals):8.6f}")
    print(f"Kurtosis:      {kurtosis(residuals):8.6f}")
    
    return residuals

# Oil blending physics validation
def validate_blending_physics(component_fractions, predicted_properties):
    """Validate predictions against basic blending physics"""
    
    # Check that component fractions sum to 1 (approximately)
    fraction_sums = component_fractions.sum(axis=1)
    fraction_violations = np.sum(np.abs(fraction_sums - 1.0) > 0.01)
    
    # Check for conservation of mass principles
    conservation_score = np.mean(np.abs(fraction_sums - 1.0))
    
    print("Blending Physics Validation:")
    print("-" * 30)
    print(f"Fraction sum violations: {fraction_violations} out of {len(fraction_sums)}")
    print(f"Average deviation from unity: {conservation_score:.6f}")
    
    return fraction_violations, conservation_score

print("Advanced analysis functions ready!")
print("\nTo use these functions after training:")
print("1. oil_property_validation(final_preds, TARGETS)")
print("2. analyze_feature_interactions(X_train_selected, y_train.iloc[:, 0])")
print("3. validate_blending_physics(train[[f'Component{i}_fraction' for i in range(1, 6)]], final_preds)")

# Optuna study continuation (for further optimization)
def continue_optimization(study, objective_func, n_additional_trials=50):
    """Continue optimization with more trials"""
    print(f"Continuing optimization with {n_additional_trials} additional trials...")
    study.optimize(objective_func, n_trials=n_additional_trials)
    print(f"Best score after additional trials: {study.best_value:.6f}")
    return study.best_params

print("\nReady for enhanced oil property prediction!")
print("All advanced features, models, and analysis tools are prepared.")